In [1]:
import cv2
import numpy as np
from PIL import Image, ImageOps

def preprocess_bw(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    _, thresh = cv2.threshold(blurred, 127, 255, cv2.THRESH_BINARY)
    return thresh

def pad_image(image, border):
    # Ensure image is uint8
    if image.dtype != np.uint8:
        image = np.clip(image, 0, 255).astype(np.uint8)

    # Skip if image is too small
    if image.shape[0] < 2 or image.shape[1] < 2:
        print(f"⚠️ Skipping padding: image too small ({image.shape})")
        return image

    try:
        pil_img = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        padded = ImageOps.expand(pil_img, border=border, fill='white')
        padded_np = np.array(padded)
        # Ensure final padded image is uint8
        padded_np = np.clip(padded_np, 0, 255).astype(np.uint8)
        return cv2.cvtColor(padded_np, cv2.COLOR_RGB2BGR)
    except Exception as e:
        print(f"❌ pad_image error: {e}")
        return image


def crop_image(image, crop_border):
    h, w = image.shape[:2]
    return image[crop_border:h - crop_border, crop_border:w - crop_border]

def rotate_image(image, angle, size):
    M = cv2.getRotationMatrix2D((size[0] / 2, size[1] / 2), angle, 1)
    return cv2.warpAffine(image, M, size, borderValue=(255, 255, 255))



def augment_image(image):
    rows, cols, _ = image.shape
    target_size = (cols, rows)

    # Base augmentations
    rotated_pos15 = rotate_image(image, 15, target_size)
    rotated_neg15 = rotate_image(image, -15, target_size)

    scaled = cv2.resize(image, None, fx=1.2, fy=1.2)
    scaled = cv2.resize(scaled, target_size, interpolation=cv2.INTER_AREA)

    M_translate = np.float32([[1, 0, 20], [0, 1, 30]])
    translated = cv2.warpAffine(image, M_translate, target_size, borderValue=(255, 255, 255))

    noise_std = 15
    noise = np.random.normal(0, noise_std, image.shape).astype(np.int16)
    noisy_image = np.clip(image.astype(np.int16) + noise, 0, 255).astype(np.uint8)



    padded_30 = cv2.resize(pad_image(image, 30), target_size)
    padded_60 = cv2.resize(pad_image(image, 60), target_size)
    padded_30_rot15 = rotate_image(padded_30, 15, target_size)
    padded_60_rot_neg15 = rotate_image(padded_60, -15, target_size)

    cropped_20 = cv2.resize(crop_image(image, 20), target_size)
    cropped_40 = cv2.resize(crop_image(image, 40), target_size)
    cropped_50 = cv2.resize(crop_image(image, 50), target_size)
    cropped_20_rot15 = rotate_image(cropped_20, 15, target_size)
    cropped_20_rot_neg15 = rotate_image(cropped_20, -15, target_size)

    images = [
        image,
        rotated_pos15,
        rotated_neg15,
        scaled,
        translated,
        noisy_image,
        
        padded_30,
        padded_60,
        padded_30_rot15,
        padded_60_rot_neg15,
        cropped_20,
        cropped_40,
        cropped_50,
        cropped_20_rot15,
        cropped_20_rot_neg15
    ]

    bw_images = [preprocess_bw(img) for img in images]

    bw_rgb_images = [cv2.cvtColor(bw_img, cv2.COLOR_GRAY2RGB) for bw_img in bw_images]

    # Convert all images to uint8 (very important)
    bw_rgb_images = [img.astype(np.uint8) for img in bw_rgb_images]

    return bw_rgb_images



In [ ]:
import numpy as np
import os
import joblib
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import MobileNetV2

# Assuming you have these functions defined elsewhere
# from your_utils import load_dataset_kaggle, load_images_with_preprocessing, augment_image

def process_and_train(dataset_num):
    data_dir = f'/kaggle/input/lettereras/{dataset_num}'
    
    if not os.path.exists(data_dir):
        return  # skip if folder doesn't exist
    
    try:
        # Load and preprocess data
        X, y = load_images_with_preprocessing(data_dir)
        
        # Data augmentation
        X_aug = []
        y_aug = []
        for img, label in zip(X, y):
            augmented_images = augment_image(img)
            for aug_img in augmented_images:
                X_aug.append(aug_img)
                y_aug.append(label)
        
        X_aug = np.array(X_aug)
        y_aug = np.array(y_aug)
        
        # Encode labels
        label_encoder = LabelEncoder()
        y_encoded = label_encoder.fit_transform(y_aug)
        y_categorical = to_categorical(y_encoded)
        
        # Save label encoder
        joblib.dump(label_encoder, f'/kaggle/working/label_encoder_{dataset_num}.pkl')

        
        #X_aug = (X_aug / 255).astype(np.float32)
        print(X_aug.shape)
        
        # Train-test split
        X_train, X_val, y_train, y_val = train_test_split(
            X_aug, y_categorical, test_size=0.1, 
            stratify=y_encoded, random_state=36
        )
        
        # Build model
        base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

        for layer in base_model.layers:
            layer.trainable = False
        
        x = base_model.output
        x = GlobalAveragePooling2D()(x)
        x = Dropout(0.5)(x)
        x = Dense(128, activation='relu')(x)
        x = Dropout(0.3)(x)
        predictions = Dense(y_categorical.shape[1], activation='softmax')(x)
        
        model = Model(inputs=base_model.input, outputs=predictions)
        model.compile(
            optimizer=Adam(learning_rate=1e-3),
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )
        
        # Train model (no logs)
        model.fit(
            X_train, y_train,
            validation_data=(X_val, y_val),
            epochs=50,
            batch_size=8,
            verbose=1  # hide training logs
        )
        
        # Save model
        model.save(f'/kaggle/working/model_{dataset_num}.h5')
        print(f"✅ Done training & saving dataset {dataset_num}")

    except Exception as e:
        print(f"❌ Error with dataset {dataset_num}: {str(e)}")

# Process all datasets from 0 to 36
for i in range(37):  # 0 to 36 inclusive
    process_and_train(i)

print("🎉 All datasets processed successfully!")
